# Training our model
This code trains a model for predicting tweet sentiment using PyTorch, Pandas and the Sentiment140 dataset.

(**NOTE:** This code is N/A, we are using TensorFlow. See `tf_train.ipynb` for latest code.)

In [83]:
import torch
import pandas as pd

In [84]:
torch.__version__

'1.13.1'

In [85]:
# Enable CUDA if available (GPU acceleration)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
device

device(type='cuda')

## Formatting data

In [86]:
# Load data
input_file = 'data/training.1600000.processed.noemoticon.csv'
df = pd.read_csv(input_file, header = None, encoding='latin1')

In [87]:
df.shape

(1600000, 6)

In [88]:
df[0].value_counts()

0    800000
4    800000
Name: 0, dtype: int64

In [89]:
'''
    Columns are as follows:
    0. target: the polarity of the tweet (0 = negative, 4 = positive)
    1. ids: The id of the tweet
    2. date: the date of the tweet
    3. flag: The query. If there is no query, then this value is NO_QUERY.
    4. user: the user that tweeted
    5. text: the text of the tweet

'''
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [90]:
df.tail()

,0,1,2,3,4,5
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


## Training model

In [91]:
import torchtext
from torchtext import data
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [92]:
torchtext.__version__

'0.14.1'

### Tokenization
Preprocessing tweets with the SpaCy tokenizer to remove unnecessary parts, keeping only meaningful words/characters

In [99]:
import re
from nltk.corpus import stopwords
from collections import Counter

def tokenize(x_train,y_train,x_val,y_val):
    word_list = []

    stop_words = set(stopwords.words('english')) 
    for sent in x_train:
        for word in sent.lower().split():
            word = preprocess_string(word)
            if word not in stop_words and word != '':
                word_list.append(word)
  
    corpus = Counter(word_list)
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}
    
    final_list_train,final_list_test = [],[]
    for sent in x_train:
            final_list_train.append([onehot_dict[preprocess_string(word)] for word in sent.lower().split() 
                                     if preprocess_string(word) in onehot_dict.keys()])
    for sent in x_val:
            final_list_test.append([onehot_dict[preprocess_string(word)] for word in sent.lower().split() 
                                    if preprocess_string(word) in onehot_dict.keys()])
            
    encoded_train = [1 if label =='positive' else 0 for label in y_train]  
    encoded_test = [1 if label =='positive' else 0 for label in y_val] 
    return np.array(final_list_train), np.array(encoded_train),np.array(final_list_test), np.array(encoded_test),onehot_dict

df_train = df[[0, 5]].copy().rename(columns={0:'sentiment', 5:'text'}) # Remove unnecessary columns and rename appropriately

regex_mentions = r"@[A-Za-z0-9_]+" # mentions
regex_links = r"https?://[A-Za-z0-9./]+" # links
regex_special = r"[^A-Za-z0-9]+" # special chars

df_train.text = df_train.text.apply(lambda x: re.sub(regex_mentions, " ", str(x).strip()))
df_train.text = df_train.text.apply(lambda x: re.sub(regex_links, " ", str(x).strip()))
df_train.text = df_train.text.apply(lambda x: re.sub(regex_special, " ", str(x).strip()))

x,y = df_train['text'].values, df_train['sentiment'].values # train-test split, x is input and y is output
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y)
x_train,y_train,x_test,y_test,vocab = tokenize(x_train,y_train,x_test,y_test)

/tmp/ipykernel_19112/53350004.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(final_list_train), np.array(encoded_train),np.array(final_list_test), np.array(encoded_test),onehot_dict


In [100]:
print(f'Length of vocabulary is {len(vocab)}')

Length of vocabulary is 1000


In [120]:
print(len(x_test))

400000


In [115]:
print(y_test[int(len(y_test)/2)])

0


In [101]:
df_train.head() # Results

,sentiment,text
0,0,Awww that s a bummer You shoulda got David Ca...
1,0,is upset that he can t update his Facebook by ...
2,0,I dived many times for the ball Managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it s not behaving at all i m mad why am i h...


In [ ]:
print(f"Null values: \n\n{df_train.isna().sum()}")

Null values: 

sentiment    0
text         0
dtype: int64


### Prepare GLoVe word embedding vectors

In [ ]:
embeddings_dictionary = dict()
embedding_dimension = 100
glove_file = open("data/glove.6B.50d.txt")

# Iterate through the glove file
for line in glove_file:
    # Split each line
    records = line.split()
    # Get the actual word
    word = records[0]
    # Get the dimensional representation
    representation = np.asarray(records[1:], dtype="float32")
    # Update the words dictionary 
    embeddings_dictionary[word] = representation
    
# Close the file
glove_file.close()

# Initialize the embedding_matrix
embeddings_matrix = np.zeros((vocab_size, embedding_dimension))

# Iterate through the tokenizer words list
for word, index in tokenizer.word_index.items():
    # Get the word representation
    embedding_vector = embeddings_dictionary.get(word)
    # Update the word representation if it exist in our dictionary
    if embedding_vector is not None:
        embeddings_matrix[index] = embedding_vector

### Define the model

In [ ]:
import torch.nn

def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim